# Hunting for Lunar Anomalies with DBSCAN

**The Outlier Detective: Finding the Moon's Hidden Treasures**

---

## The Hook

Most algorithms look for the **average**. Today, we're going to look for the **weird stuff**.

Traditional clustering (like K-Means) tries to group everything into neat categories. But what about those oddball points that don't fit anywhere? Those are often the most scientifically interesting!

**We're going to use machine learning to find geochemical outliers that might represent:**
- **Fresh impact craters** (like Tycho) that have excavated unique material
- **Pyroclastic deposits** (like the Aristarchus Plateau) from ancient volcanic eruptions
- **Data artifacts** or instrument anomalies worth investigating
- **Undiscovered Points of Interest** for future lunar missions

Think of this as a **treasure hunt**, we're using ML to flag locations that deserve a closer look!

---

## 1. Import Packages

We'll use the same tools as before, plus a new algorithm:
- **Polars**: Fast DataFrame library for data manipulation
- **Matplotlib**: For creating visualizations
- **Scikit-learn**: For DBSCAN and data preprocessing

In [ ]:
# Data manipulation library
import polars as pl

# Visualization library
import matplotlib.pyplot as plt
from matplotlib.patches import Rectangle
from matplotlib.collections import PatchCollection

# Machine learning tools
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import DBSCAN

---

## 2. Load the Lunar Prospector GRS Data

We'll use the same data loading function from our previous workshop file. This reads the Lunar Prospector Gamma-Ray Spectrometer elemental abundance data.

In [ ]:
def load_lpgrs_tab(filepath: str) -> pl.DataFrame:
    """
    Loads a Lunar Prospector GRS .tab file into a Polars DataFrame.
    
    Handles wrapped lines, multiple-space delimiters, and scientific notation.

    Args:
        filepath (str): Path to the .tab file.

    Returns:
        pl.DataFrame: DataFrame containing the GRS data with appropriate column names.
    """
    # Read all whitespace-separated tokens (NASA records wrap across lines)
    with open(filepath, 'r') as f:
        tokens = f.read().split()
    
    # Reshape into 61 columns (standard GRS format)
    record_width = 61
    rows = [tokens[i : i + record_width] for i in range(0, len(tokens), record_width)]
    
    # Create DataFrame and cast to Float64
    df = pl.DataFrame(rows, orient="row").select([
        pl.all().cast(pl.Float64)
    ])
    
    # Human-readable column names for key measurements
    names = [
        "bin_index",    # Unique identifier for each spatial bin
        "lat_start",    # Starting latitude of the bin (degrees)
        "lat_end",      # Ending latitude of the bin (degrees)
        "lon_start",    # Starting longitude of the bin (degrees)
        "lon_end",      # Ending longitude of the bin (degrees)
        "thorium",      # Thorium abundance (ppm) - key KREEP indicator
        "th_err",       # Thorium measurement uncertainty
        "potassium",    # Potassium abundance (wt%) - key KREEP indicator
        "k_err",        # Potassium measurement uncertainty
        "iron",         # Iron abundance (wt%) - high in mare basalts
        "fe_err",       # Iron measurement uncertainty
        "titanium",     # Titanium abundance (wt%) - high in high-Ti mare basalts
        "ti_err",       # Titanium measurement uncertainty
        "samarium",     # Samarium abundance (ppm) - rare earth element
        "sm_err",       # Samarium measurement uncertainty
        "calcium"       # Calcium abundance (wt%) - high in anorthosites
    ]
    
    return df.select(df.columns[:16]).rename(
        {old: new for old, new in zip(df.columns[:16], names)}
    )

# Load the data
path_to_tab = "data/lpgrs_high1_elem_abundance_5deg.tab"
df = load_lpgrs_tab(path_to_tab)

print(f"Loaded {df.height} records with {df.width} columns")
df.head()

---

## 3. Data Preparation

As before, we need to:
1. Select our geochemical features
2. Clean any null values
3. Standardize the data so all elements contribute equally

In [ ]:
# Select geochemical features for analysis
feature_cols = ['iron', 'titanium', 'thorium', 'potassium']

# Clean the data - remove any rows with null values
df_clean = df.drop_nulls(subset=feature_cols)

print(f"Clean dataset size: {df_clean.height} rows")

# Standardize the data
# This is CRITICAL - DBSCAN uses distance, so we need all features on the same scale
scaler = StandardScaler()
X_scaled = scaler.fit_transform(df_clean.select(feature_cols).to_numpy())

print("Data standardized successfully!")

---

## 4. Introducing DBSCAN: The Outlier Detector

### What is DBSCAN?

**DBSCAN** (Density-Based Spatial Clustering of Applications with Noise) is fundamentally different from K-Means:

| K-Means | DBSCAN |
|---------|--------|
| Groups everything into k clusters | Finds clusters of any shape |
| Every point must belong to a cluster | **Points can be labeled as "Noise" (-1)** |
| You specify number of clusters | Discovers clusters automatically |
| Struggles with outliers | **Designed to find outliers!** |

### How DBSCAN Works

DBSCAN has two key parameters:

1. **`eps`** (epsilon): The maximum distance between two points to be considered neighbors
2. **`min_samples`**: The minimum number of points to form a dense region

The algorithm classifies each point as:
- **Core Point**: Has at least `min_samples` neighbors within `eps` distance
- **Border Point**: Within `eps` of a core point but doesn't have enough neighbors itself
- **Noise Point**: Neither core nor border — **these are our outliers!** (labeled as -1)

### Our Strategy: Be Strict!

By setting a **small `eps`** value, we're saying: "Only group points that are *very* similar together."

This means more points will be flagged as **"Noise"**, exactly what we want for finding unusual geochemical signatures!

In [ ]:
# Configure DBSCAN with strict parameters
# eps=0.5 means points must be within 0.5 standard deviations to be neighbors
# min_samples=5 means we need at least 5 nearby points to form a cluster

dbscan = DBSCAN(
    eps=0.5,           # Distance threshold (in standardized units)
    min_samples=5      # Minimum points to form a dense region
)

# Fit DBSCAN to our data
labels = dbscan.fit_predict(X_scaled)

# Add the cluster labels to our DataFrame
df_result = df_clean.with_columns(
    pl.Series("cluster", labels)
)

# Count the results
unique_labels = set(labels)
n_clusters = len(unique_labels) - (1 if -1 in unique_labels else 0)
n_noise = list(labels).count(-1)

print(f"DBSCAN Results:")
print(f"  - Number of clusters found: {n_clusters}")
print(f"  - Number of NOISE points (outliers): {n_noise}")
print(f"  - Percentage of data flagged as outliers: {100 * n_noise / len(labels):.1f}%")

---

## 5. Visualize All Data with Cluster Labels

First, let's see the full picture. All points colored by their cluster assignment. Points labeled `-1` (Noise) will stand out!

In [ ]:
# Create a map showing all data points with cluster labels
fig, ax = plt.subplots(figsize=(14, 7))

# Calculate bin center coordinates for plotting
lats = (df_result["lat_start"].to_numpy() + df_result["lat_end"].to_numpy()) / 2
lons = (df_result["lon_start"].to_numpy() + df_result["lon_end"].to_numpy()) / 2
clusters = df_result["cluster"].to_numpy()

# Create a colormap, noise points (-1) will be bright red
cmap = plt.cm.viridis
colors = []
for c in clusters:
    if c == -1:
        colors.append('red')  # Noise points in red
    else:
        colors.append(cmap(c / max(1, max(clusters))))

# Plot all points
scatter = ax.scatter(lons, lats, c=colors, s=15, alpha=0.7)

# Add title and labels
ax.set_xlabel("Longitude (°)", fontsize=12)
ax.set_ylabel("Latitude (°)", fontsize=12)
ax.set_title("DBSCAN Clustering of Lunar GRS Data\n(Red = Noise/Outliers)", fontsize=14)
ax.set_xlim(-180, 180)
ax.set_ylim(-90, 90)
ax.grid(True, alpha=0.3)
ax.set_aspect('equal')

plt.tight_layout()
plt.show()

print("Red points = Outliers (Noise) — These are our targets!")

---

## 6. The Treasure Map: Isolating the Outliers

Now for the exciting part! Let's create a map showing **ONLY** the 215 outlier points!

These are the locations where the geochemistry is unusual enough that DBSCAN couldn't group them with the "normal" lunar surface. We'll color them by Thorium content to see the KREEP signature.

In [ ]:
# Filter to get ONLY the outlier points (cluster label = -1)
df_outliers = df_result.filter(pl.col("cluster") == -1)

print(f"Found {df_outliers.height} outlier points to investigate!")
print(f"\nThese represent {100 * df_outliers.height / df_result.height:.1f}% of the lunar surface")

# Create the "Treasure Map" - showing only outliers
fig, ax = plt.subplots(figsize=(14, 7))

# Calculate coordinates for outliers
outlier_lats = (df_outliers["lat_start"].to_numpy() + df_outliers["lat_end"].to_numpy()) / 2
outlier_lons = (df_outliers["lon_start"].to_numpy() + df_outliers["lon_end"].to_numpy()) / 2

# Color outliers by their Thorium content (a key indicator of interesting geology)
thorium_values = df_outliers["thorium"].to_numpy()

# Create rectangles for each 5-degree bin
patches = []
for i in range(len(df_outliers)):
    row = df_outliers.row(i, named=True)
    rect = Rectangle(
        (row["lon_start"], row["lat_start"]),
        row["lon_end"] - row["lon_start"],
        row["lat_end"] - row["lat_start"]
    )
    patches.append(rect)

# Create the patch collection with Thorium-based coloring
collection = PatchCollection(patches, cmap='hot', alpha=0.8)
collection.set_array(thorium_values)
ax.add_collection(collection)

# Add colorbar
cbar = plt.colorbar(collection, ax=ax, label='Thorium (ppm)')

# Styling
ax.set_xlabel("Longitude (°)", fontsize=12)
ax.set_ylabel("Latitude (°)", fontsize=12)
ax.set_title("Lunar Anomaly Treasure Map\nGeochemical Outliers Detected by DBSCAN\n(Color = Thorium Concentration)", 
             fontsize=14)
ax.set_xlim(-180, 180)
ax.set_ylim(-90, 90)
ax.grid(True, alpha=0.3, linestyle='--')
ax.set_aspect('equal')
ax.set_facecolor('lightgray')

plt.tight_layout()
plt.show()

---

## 7. Investigate the Outliers: What Makes Them Special?

Let's compare the geochemical properties of our 215 outliers to the 1,576 "normal" points. Are outliers enriched or depleted in certain elements?

In [ ]:
# Compare outliers to normal points
df_normal = df_result.filter(pl.col("cluster") != -1)

print("=" * 60)
print("GEOCHEMICAL COMPARISON: Outliers vs. Normal Lunar Surface")
print("=" * 60)

for col in feature_cols:
    outlier_mean = df_outliers[col].mean()
    normal_mean = df_normal[col].mean()
    outlier_std = df_outliers[col].std()
    
    # Calculate how many standard deviations the outlier mean is from normal
    difference = (outlier_mean - normal_mean) / df_normal[col].std() if df_normal[col].std() > 0 else 0
    
    direction = "↑ HIGHER" if outlier_mean > normal_mean else "↓ LOWER"
    
    print(f"\n{col.upper()}:")
    print(f"  Normal surface mean: {normal_mean:.3f}")
    print(f"  Outlier mean:        {outlier_mean:.3f}")
    print(f"  Difference:          {direction} by {abs(difference):.1f}σ")

print("\n" + "=" * 60)

---

## 8. Experiment: Tuning the Sensitivity

The `eps` parameter controls how "picky" DBSCAN is. Let's see how changing it affects our outlier detection:

- **Smaller eps** → More strict → More outliers detected
- **Larger eps** → More lenient → Fewer outliers detected

In [ ]:
# Test different eps values
eps_values = [0.3, 0.5, 0.7, 1.0, 1.5]

fig, axes = plt.subplots(1, len(eps_values), figsize=(20, 4))

print("Effect of eps parameter on outlier detection:")
print("-" * 50)

for idx, eps in enumerate(eps_values):
    # Run DBSCAN with this eps value
    dbscan_test = DBSCAN(eps=eps, min_samples=5)
    test_labels = dbscan_test.fit_predict(X_scaled)
    
    # Count outliers
    n_outliers = list(test_labels).count(-1)
    pct_outliers = 100 * n_outliers / len(test_labels)
    
    print(f"eps = {eps}: {n_outliers} outliers ({pct_outliers:.1f}%)")
    
    # Create subplot
    ax = axes[idx]
    
    # Plot all points, highlight outliers
    colors = ['red' if l == -1 else 'lightblue' for l in test_labels]
    ax.scatter(lons, lats, c=colors, s=5, alpha=0.6)
    
    ax.set_title(f"eps = {eps}\n{n_outliers} outliers ({pct_outliers:.1f}%)")
    ax.set_xlim(-180, 180)
    ax.set_ylim(-90, 90)
    ax.set_aspect('equal')
    ax.grid(True, alpha=0.3)

print("-" * 50)
plt.suptitle("Sensitivity Analysis: How eps Affects Outlier Detection", fontsize=14, y=1.05)
plt.tight_layout()
plt.show()

---

## 9. Top Outlier Locations

Let's rank the outliers by how "extreme" their geochemistry is. We'll compute an extremeness score (sum of absolute standardized values) to find the most unusual points on the Moon.

In [ ]:
# Sort outliers by their most extreme geochemical values
# We'll use a composite "extremeness" score based on standardized values

# Get scaled values for outliers
outlier_indices = [i for i, l in enumerate(labels) if l == -1]
X_outliers = X_scaled[outlier_indices]

# Calculate how extreme each outlier is (sum of absolute standardized values)
extremeness = [sum(abs(x)) for x in X_outliers]

# Add extremeness to outlier dataframe
df_outliers_ranked = df_outliers.with_columns(
    pl.Series("extremeness", extremeness)
).sort("extremeness", descending=True)

# Display top 10 most extreme outliers
print("TOP 10 MOST EXTREME OUTLIERS")
print("=" * 80)
print(f"{'Rank':<6} {'Lat':<10} {'Lon':<10} {'Fe':<8} {'Ti':<8} {'Th':<8} {'K':<8} {'Score':<8}")
print("-" * 80)

for i in range(min(10, df_outliers_ranked.height)):
    row = df_outliers_ranked.row(i, named=True)
    lat = (row["lat_start"] + row["lat_end"]) / 2
    lon = (row["lon_start"] + row["lon_end"]) / 2
    print(f"{i+1:<6} {lat:>7.1f}°  {lon:>7.1f}°  {row['iron']:<8.2f} {row['titanium']:<8.2f} "
          f"{row['thorium']:<8.2f} {row['potassium']:<8.3f} {row['extremeness']:<8.2f}")

print("=" * 80)
print("\nCompare these coordinates to known lunar features!")

---

## 10. Interpreting Our Results

### What Did We Find?

Looking at the top outliers, we see a striking pattern:

| Location | Coordinates | What's There |
|----------|-------------|--------------|
| **Oceanus Procellarum** | 20°N, -57°W | Highest extremeness score! The KREEP-rich region |
| **Aristarchus Region** | 15-25°N, -47°W | Pyroclastic deposits, high Th/Ti |
| **Mare Imbrium Edge** | 10°N, 22-27°E | Boundary of major basin |
| **Copernicus Area** | 20°N, -32°W | Near the bright-rayed crater |

**The Procellarum KREEP Terrane (PKT)** dominates our outlier map! This is the most geochemically distinct region on the Moon, enriched in Thorium, Potassium, and rare earth elements.

### Geochemical Signatures of Outliers

Our comparison showed outliers have:
- **Higher Titanium** (+4.0σ): indicating high-Ti mare basalts
- **Higher Thorium** (+3.3σ): the classic KREEP signature
- **Higher Potassium** (+1.1σ): another KREEP indicator
- **Lower Iron** (-2.3σ): less mafic than typical mare basalts

This chemical fingerprint points to **evolved, KREEP-rich materials**. Exactly what geologists expect in the PKT!

### Notable Lunar Features Reference

| Feature | Latitude | Longitude | Connection to Our Outliers |
|---------|----------|-----------|---------------------------|
| **Aristarchus Plateau** | 24°N | -47°W | Multiple outliers here! |
| **Oceanus Procellarum** | 18°N | -57°W | Highest extremeness scores |
| **Copernicus Crater** | 10°N | -20°W | Nearby outliers detected |
| **Tycho Crater** | -43°S | -11°W | Some scattered outliers in farside |
| **South Pole-Aitken Basin** | -53°S | 169°W | Sparse outliers on farside |

### Discussion Questions:

1. **Why does the PKT dominate our outliers?** What makes this region so chemically distinct from the rest of the Moon?
2. **Notice the farside outliers**: They're scattered and isolated. What might cause these?
3. **The high-Ti signature**: Where do high-Ti basalts come from on the Moon?
4. **Could any outliers be data artifacts?** How would you tell the difference?

---

## Summary: What We Learned

### Our Key Results

With `eps=0.5` and `min_samples=5`, DBSCAN identified:
- **6 distinct clusters** of "normal" lunar geochemistry
- **215 outlier points (12%)** flagged as geochemically unusual
- **The outliers concentrate in the Procellarum KREEP Terrane** (the Moon's most evolved volcanic province)

### DBSCAN vs. K-Means

| Aspect | K-Means | DBSCAN |
|--------|---------|--------|
| Goal | Classify everything | Find dense regions + outliers |
| Output | Every point gets a label | Some points are "Noise" |
| Parameters | Number of clusters (k) | Distance (eps), density (min_samples) |
| Best for | Understanding the average | Finding the unusual |

### The Power of Sensitivity Tuning

Our eps experiment showed:
- **eps = 0.3**: 598 outliers (33%): too aggressive, catches too much
- **eps = 0.5**: 215 outliers (12%): good balance
- **eps = 0.7**: 63 outliers (3.5%): only the most extreme
- **eps = 1.0**: 15 outliers (0.8%): just the rarest anomalies
- **eps = 1.5**: 0 outliers: everything grouped together

### Why This Matters for Lunar Science

DBSCAN revealed that the **Procellarum KREEP Terrane** stands out as geochemically anomalous, a finding that aligns with decades of lunar research. This region:

1. Contains the Moon's highest Thorium concentrations
2. Hosts ancient volcanic centers (Aristarchus Plateau)
3. May represent the last remnants of the lunar magma ocean
4. Is a prime target for future sample return missions

### Key Takeaways

- **Outliers can be scientifically important**: our "noise" points mapped the PKT!
- **Parameter tuning is critical**: eps of 0.5 gave us 12% outliers; 1.5 gave us none
- **ML validates known geology**: we independently rediscovered the PKT
- **Automated discovery**: this approach could find unknown anomalies on other planetary bodies

---

## Next Steps

1. Try `min_samples=3` or `min_samples=10`: how does cluster structure change?
2. Add Samarium and Calcium to the features: do new outliers appear?
3. Run DBSCAN on **just the farside**: what stands out when PKT is excluded?
4. Export outlier coordinates and overlay on an image mosaic
5. Apply this technique to Mars GRS data!